In [1]:
import os,sys
b_directory = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'model'))
sys.path.insert(0, b_directory)
from pan_epitope_single import *
import numpy as np
import pandas as pd
import torch,random
from torch import nn
from torch.optim import Adam
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import (
    average_precision_score,
    roc_auc_score
)
import warnings
warnings.filterwarnings("ignore")

In [2]:
class sx_Dataset(Dataset):
    def __init__(self,data1,data2,data3,):
        self.x1 = data1
        self.x2 = data2
        self.x3 = data3

        self.len = data1.shape[0]
 
    def __getitem__(self, index):
        return self.x1[index],self.x2[index],self.x3[index]
 
    def __len__(self):
        return self.len

In [3]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [4]:
class sx_Dataset(Dataset):
    def __init__(self,data1,data2,data3,):
        #ep emb, pos_emb,
        self.x1 = data1
        self.x2 = data2
        self.x3 = data3

        self.len = data1.shape[0]
 
    def __getitem__(self, index):
        return self.x1[index],self.x2[index],self.x3[index]
 
    def __len__(self):
        return self.len

In [5]:
def cal_(beta_train_emb,ep_train_emb,train_health_tcr_emb,seed=1,lr=0.0001,EPOCH=20,BATCH_SIZE=64,device='cuda:0',save_path='./model.pt'):
    set_seed(seed)
    
    model=classification_model(tcr_dim=beta_train_emb.shape[-1], pep_dim=ep_train_emb.shape[-1], 
                               )
    model=model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    train_neg_nums=beta_train_emb.shape[0]
    
    for epoch in range(EPOCH):
        model.train()       
        random.seed(epoch)

        health_nums=random.randint(0,train_health_tcr_emb.shape[0]-train_neg_nums-1)
        dataset = sx_Dataset(beta_train_emb,train_health_tcr_emb[health_nums:health_nums+train_neg_nums],
                             ep_train_emb)
        train_dataloader=DataLoader(dataset=dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=4,drop_last=True)
        for tra_step, (cdr3_pos,Hcdr3_pos,pep) in enumerate(train_dataloader):   
            pep=torch.tensor(pep,dtype=torch.float32).to(device) 
            cdr3_pos=torch.tensor(cdr3_pos,dtype=torch.float32).to(device)    
            Hcdr3_pos=torch.tensor(Hcdr3_pos,dtype=torch.float32).to(device) 

            pos_pred = model(cdr3_pos,pep)
            pos_pred = pos_pred.flatten()
            
            health_pred = model(Hcdr3_pos,pep)
            health_pred = health_pred.flatten()
                  
            pos_loss = F.binary_cross_entropy(pos_pred,torch.ones_like(pos_pred).to(device))
            health_loss = F.binary_cross_entropy(health_pred,torch.zeros_like(health_pred).to(device))
            
            loss = pos_loss+health_loss
            optimizer.zero_grad()
            loss.requires_grad_(True)
            loss.backward()
            optimizer.step()
    torch.save(model,save_path)

In [6]:
beta_train_emb=np.load('../../tmp_data/2/train_cdr3_emb.npy')
ep_train_emb=np.load('../../tmp_data/2/ep_train_emb.npy')
train_health_tcr_emb=np.load('../../tmp_data/2/health_tcr_emb.npy')
model_save_path='./model.pt'
cuda='cuda:0'

In [7]:
cal_(beta_train_emb,ep_train_emb,train_health_tcr_emb,device=cuda,save_path=model_save_path)

In [8]:
model=torch.load(model_save_path)

In [9]:
model.eval()

classification_model(
  (cdr3_linear): Linear(in_features=1024, out_features=256, bias=True)
  (pep_linear): Linear(in_features=1024, out_features=256, bias=True)
  (inter_layers): ModuleList(
    (0-1): 2 x Sequential(
      (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (seqlevel_outlyer): Sequential(
    (0): AdaptiveMaxPool2d(output_size=1)
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=1, bias=True)
    (4): Sigmoid()
  )
)

In [10]:
test_cdr3_emb=np.load('../../tmp_data/2/test_cdr3_emb.npy')
ep_test_emb=np.load('../../tmp_data/2/ep_test_emb.npy')
test_labels=pd.read_csv('../../tmp_data/2/test.csv')['Label'].to_numpy()

In [11]:
preds=model(torch.tensor(test_cdr3_emb,dtype=torch.float32).to(cuda),
            torch.tensor(ep_test_emb,dtype=torch.float32).to(cuda))

In [12]:
roc_auc_score(test_labels,preds.detach().cpu().numpy())

0.9041322314049587